# Tweem.io Similarity Comparison

### Imports

In [2]:
#  Necessary to allow assembly framework to run
import os
os.chdir('..')

In [3]:
import pandas as pd
import numpy as np
import inspect

import assembly
from assembly import config as asmbl_config
from assembly import db as asmbl_db
from assembly import models as asmbl_models

from lib.tweemio import twm
from lib.tweemio import similarity
from lib.tweemio import twitter

### Settings

In [4]:
os.environ['ASSEMBLY_ENV'] = 'Development'
app = assembly.Assembly.init(__name__, {'default': []})

### Calculate Similarity to Group

In [9]:
results = twm.calculate(app.config, screen_name='barackobama', group='comedians') #, compare_to_screen_names=['realdonaldtrump'])

Evaluating similarity to billmaher with model: mdl_multinomialnbvect_v1
Evaluating similarity to conanobrien with model: mdl_multinomialnbvect_v1
Evaluating similarity to jimmyfallon with model: mdl_multinomialnbvect_v1
Evaluating similarity to jimmykimmel with model: mdl_multinomialnbvect_v1
Evaluating similarity to trevornoah with model: mdl_multinomialnbvect_v1
Evaluating similarity to jimgaffigan with model: mdl_multinomialnbvect_v1
Evaluating similarity to sarahksilverman with model: mdl_multinomialnbvect_v1
Evaluating similarity to stevemartintogo with model: mdl_multinomialnbvect_v1
Evaluating similarity to stephenathome with model: mdl_multinomialnbvect_v1
Evaluating similarity to badbanana with model: mdl_multinomialnbvect_v1
Evaluating similarity to jerryseinfeld with model: mdl_multinomialnbvect_v1
Evaluating similarity to sethmeyers with model: mdl_multinomialnbvect_v1
Evaluating similarity to theellenshow with model: mdl_multinomialnbvect_v1


In [10]:
{screen_name : np.mean(result.y_prob) for screen_name, result in results.items()}

{'billmaher': 0.2982940738047731,
 'conanobrien': 0.01455667571280429,
 'jimmyfallon': 0.003567402346777809,
 'jimmykimmel': 0.0017560271154860086,
 'trevornoah': 0.041832991138385865,
 'jimgaffigan': 0.0021435156058362526,
 'sarahksilverman': 0.022085711726128183,
 'stevemartintogo': 0.0026020952211481,
 'stephenathome': 0.061395777382045505,
 'badbanana': 0.022870452678794142,
 'jerryseinfeld': 0.0018900487727661903,
 'sethmeyers': 0.007672429840242245,
 'theellenshow': 0.002739452155260219}

### Scratchpad

In [119]:
screen_name='realdonaldtrump'
group='default'

In [120]:
twitter_creds = config['TWITTER_API_CREDS']
tapi = twitter.TwitterApi(twitter_creds)

#  Download user timeline;  condense / merge tweets, based on settings
print(f'Downloading user timeline for: {screen_name}')
tweet_timeline = tapi.timeline(screen_name)
timeline_text = list(reversed([tli._json['full_text'] for tli in tweet_timeline]))
timeline_text = twm.condense_timeline(timeline_text, condense_factor=config['TWEET_CONDENSE_FACTOR'])

In [121]:
tweets = timeline_text

In [122]:
#  Transform tweets to vectorizer
#  Transform incoming tweets to vectorized values
vect_mdl = similarity.get_model(grp=group, model_name='transformation-countvectorizer')
x_vect = vect_mdl.transform(tweets)

In [123]:
#  Predict probability via model
screen_mdl = similarity.get_model(grp=group, model_name='realdonaldtrump')

In [124]:
screen_mdl.classes_

array([False,  True])

In [148]:
screen_mdl.classes_

array([False,  True])

In [125]:
y_pred = screen_mdl.predict(x_vect)

In [126]:
y_prob = screen_mdl.predict_proba(x_vect)

In [127]:
pred_prob_lbl[:5]

pred_prob_opp[:5]

pred_prob_true  = y_prob[:,1]
pred_prob_false = y_prob[:,0]

np.mean(y_prob[:,1])

np.mean(y_prob[:,0])

y_pred[:10]

pred_prob_true[:10]

pred_prob_false[:10]

(~y_pred * pred_prob_false)[:5]

(y_pred * pred_prob_true)[:5]

((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))[:5]

np.mean((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))

np.std((~y_pred * (1 - pred_prob_false)) + (y_pred * pred_prob_true))

# --------------------------------------------------------

labels = {}
for screen_name in config['SIMILARITY_COMPARISONS']['default']['screen_names']:
    model = similarity.get_model('default', model_name=screen_name)
    labels[screen_name]  = model.classes_



array([4.03911071e-04, 1.45133999e-17, 3.41111708e-03, 1.27098412e-10,
       9.99984482e-01])